In [1]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

In [2]:
import ginza
from spacy import displacy
def to_dependency_data(doc):
    words = []
    arcs = []
    bunsetu_head_list = ginza.bunsetu_head_list(doc)
    for i, chunk in enumerate(ginza.bunsetu_spans(doc)):
        words.append({
            'text': chunk.text, 'tag': chunk.label_
        })
        for token in chunk.lefts:
            arcs.append({
                'start': bunsetu_head_list.index(token.i),
                'end': i,
                'label': token.dep_,
                'dir': 'left'
            })
        for token in chunk.rights:
            arcs.append({
                'start': i,
                'end': bunsetu_head_list.index(token.i),
                'label': token.dep_,
                'dir': 'right'
            })
    return {'words': words, 'arcs': arcs}


In [6]:
t = "好きなスイーツはありますか？"
t = "ダイビングしたことはありますか?"

In [7]:

displacy.render(
    to_dependency_data(nlp(t)),
    style='dep',
    jupyter=True,
    manual=True
)

In [8]:
from spacy import displacy
displacy.render(nlp(t), style='dep', jupyter=True, options={'compact':True, 'distance': 90})

In [7]:
path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [17]:
def make_Xy(convs):
    X = []
    y = []
    for conv in convs:
        for i, ut in enumerate( conv ):
            if not ut.is_system() and ut.is_type_included("質問"):
                X.append(ut.utt)
    return X

In [18]:
X= make_Xy(convs)

In [53]:
from response.feature import Feature

from sklearn.linear_model import LogisticRegression
F_path = "../X_y_data/response2/"
F_name = "Classify_F3.pickle"
featureM = DataManager(F_path)
F = featureM.load_data(F_name)

model_path = "../models/response2/"
model_name = "Classify_M3.pickle"
modelM = DataManager(model_path)
clf = modelM.load_data(model_name)

success load : ../X_y_data/response2/Classify_F3.pickle
success load : ../models/response2/Classify_M3.pickle


In [50]:
# rule = "ある　知る　覚える".split()
rule = "知る　覚える".split()

def extract_RDF_triple(text, clean_=True):
    if clean_:
        text = clean_text(text)
    doc = nlp(text)
    triple_list = []
    for _, token in enumerate( doc ):
        if token.pos_ in ["VERB", "ADJ"]:
            # print(token.lemma_)
            # "ある　知る　覚える"
            if token.lemma_ in rule:
                for c in token.children:
                    if c.dep_ in ["nsubj", "obj", "obl"]:
                        print(text, token.pos_)
                        # for c2 in c.children:
                        #     # 名詞の修飾
                        #     if c2.dep_ in ["acl"]:
                        #         print(text)

In [86]:
rule = "ある　知る　覚える".split()
def expect_rule(text, clean_=True):
    if clean_:
        text = clean_text(text)
    doc = nlp(text)
    if 1 < len(list(doc.sents)):
        doc = list(doc.sents)[-1]
        # return
    triple_list = []
    for _, token in enumerate( doc ):
        if token.pos_ in ["VERB", "ADJ"]:
            # print(token.lemma_)
            # "ある　知る　覚える"
            if token.lemma_ in rule:
                for c in token.children:
                    if c.dep_ in ["nsubj", "obj", "obl"]:
                        # print(str(doc), token.lemma_)
                        # break
                        for c2 in c.children:
                            # 名詞の修飾
                            if c2.dep_ in ["acl"]:
                                # print(str(doc), token.lemma_)
                                print(str(doc))

In [87]:
t = "学校への行き方を知っていますか？"
t = "好きなスイーツはありますか？"
extract_RDF_triple(t)

好きなスイーツはありますか? VERB


In [88]:
for x in X:
    expect_rule(x)

海外旅行されたことありますか?
この付近に美味しい御店があるかな?
あなたは嫌いな食べ物はありますか。
見たことある?
好きな映画はありますか?
他にもこんなに続いているシリーズはあるんでしょうか?
所で好きな数字ってある?
犬に何か芸を教えたことはありますか?
あなたは熱中症になったことありますか?
好きな食べ物はありますか?
ダイビングしたことはありますか?
食べたことありますか?
hikakinのどうかは見たことある?
あなたには好きな曲はありますか?
日本食で好きな物はありますか?
北海道に行ったことはありますか?
見たことありますか?
見たことありますか?
サーカー観戦したことありますか?
セブンイレブンのコーヒーは飲んだことある?
じゃあ半沢直樹見たことある?
菫味以外にどんな変わった味がありますか。
漫画喫茶みたいな所へ行ったことありますか
あなたも寝ることはあるんですか?
最近何か気になることはありますか?
好きな国はありますか?
他に好きなスポーツはありますか?
韓国料理で何か好きな物はありますか?
あなたは好きなスポーツがあるのかな?
他に得意な物はありますか?
他にゲームセンターで好きなゲームはありますか?
食べ物で好きな物はありますか?
得意な料理とかありますか?
他にはスポーツをしたことはありますか?
好きな食べ物はありますか?
好きな本はありますか?
私は、野球が好きなのですが、好きな球団はありますか?
行ったことはありますか
